# SGLang Native APIs

Apart from the OpenAI compatible APIs, the SGLang Runtime also provides its native server APIs. We introduce these following APIs:

- `/generate` (text generation model)
- `/get_model_info`
- `/get_server_info`
- `/health`
- `/health_generate`
- `/flush_cache`
- `/update_weights`
- `/encode`(embedding model)
- `/v1/rerank`(cross encoder rerank model)
- `/classify`(reward model)
- `/start_expert_distribution_record`
- `/stop_expert_distribution_record`
- `/dump_expert_distribution_record`

We mainly use `requests` to test these APIs in the following examples. You can also use `curl`.

## Launch A Server

In [1]:
import requests
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process


server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0"
)
## To run qwen2.5-0.5b-instruct model on the Ascend-Npu, you can execute the following command:
# server_process, port = launch_server_cmd(
#     "python3 -m sglang.launch_server --model-path qwen/qwen2.5-0.5b-instruct --host 0.0.0.0 --device npu --tp 2 --attention-backend torch_native"
# )

wait_for_server(f"http://localhost:{port}")

[2025-06-28 00:35:54] server_args=ServerArgs(model_path='qwen/qwen2.5-0.5b-instruct', tokenizer_path='qwen/qwen2.5-0.5b-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='qwen/qwen2.5-0.5b-instruct', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=39644, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=959176072, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=No

[2025-06-28 00:36:08] Attention backend not set. Use fa3 backend by default.
[2025-06-28 00:36:08] Init torch distributed begin.


[2025-06-28 00:36:09] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 00:36:10] Load weight begin. avail mem=63.05 GB


[2025-06-28 00:36:10] Using model weights format ['*.safetensors']
[2025-06-28 00:36:10] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


...

[2025-06-28 00:36:11] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=46.47 GB


[2025-06-28 00:36:12] INFO:     Started server process [522199]
[2025-06-28 00:36:12] INFO:     Waiting for application startup.
[2025-06-28 00:36:12] INFO:     Application startup complete.
[2025-06-28 00:36:12] INFO:     Uvicorn running on http://0.0.0.0:39644 (Press CTRL+C to quit)


[2025-06-28 00:36:12] INFO:     127.0.0.1:54152 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-28 00:36:13] INFO:     127.0.0.1:54156 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 00:36:13] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 00:36:15] INFO:     127.0.0.1:54158 - "POST /generate HTTP/1.1" 200 OK
[2025-06-28 00:36:15] The server is fired up and ready to roll!


## Generate (text generation model)
Generate completions. This is similar to the `/v1/completions` in OpenAI API. Detailed parameters can be found in the [sampling parameters](./sampling_params.md).

In [2]:
url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-06-28 00:36:17] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 00:36:17] Decode batch. #running-req: 1, #token: 40, token usage: 0.00, cuda graph: False, gen throughput (token/s): 6.42, #queue-req: 0


[2025-06-28 00:36:18] Decode batch. #running-req: 1, #token: 80, token usage: 0.00, cuda graph: False, gen throughput (token/s): 129.90, #queue-req: 0


[2025-06-28 00:36:18] Decode batch. #running-req: 1, #token: 120, token usage: 0.01, cuda graph: False, gen throughput (token/s): 131.11, #queue-req: 0
[2025-06-28 00:36:18] INFO:     127.0.0.1:47062 - "POST /generate HTTP/1.1" 200 OK


## Get Model Info

Get the information of the model.

- `model_path`: The path/name of the model.
- `is_generation`: Whether the model is used as generation model or embedding model.
- `tokenizer_path`: The path/name of the tokenizer.

In [3]:
url = f"http://localhost:{port}/get_model_info"

response = requests.get(url)
response_json = response.json()
print_highlight(response_json)
assert response_json["model_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json["is_generation"] is True
assert response_json["tokenizer_path"] == "qwen/qwen2.5-0.5b-instruct"
assert response_json.keys() == {"model_path", "is_generation", "tokenizer_path"}

[2025-06-28 00:36:18] INFO:     127.0.0.1:47066 - "GET /get_model_info HTTP/1.1" 200 OK


## Get Server Info
Gets the server information including CLI arguments, token limits, and memory pool sizes.
- Note: `get_server_info` merges the following deprecated endpoints:
  - `get_server_args`
  - `get_memory_pool_size` 
  - `get_max_total_num_tokens`

In [4]:
# get_server_info

url = f"http://localhost:{port}/get_server_info"

response = requests.get(url)
print_highlight(response.text)

[2025-06-28 00:36:18] INFO:     127.0.0.1:47072 - "GET /get_server_info HTTP/1.1" 200 OK


## Health Check
- `/health`: Check the health of the server.
- `/health_generate`: Check the health of the server by generating one token.

In [5]:
url = f"http://localhost:{port}/health_generate"

response = requests.get(url)
print_highlight(response.text)

[2025-06-28 00:36:18] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 00:36:19] INFO:     127.0.0.1:47088 - "GET /health_generate HTTP/1.1" 200 OK


In [6]:
url = f"http://localhost:{port}/health"

response = requests.get(url)
print_highlight(response.text)

[2025-06-28 00:36:19] INFO:     127.0.0.1:47104 - "GET /health HTTP/1.1" 200 OK


## Flush Cache

Flush the radix cache. It will be automatically triggered when the model weights are updated by the `/update_weights` API.

In [7]:
# flush cache

url = f"http://localhost:{port}/flush_cache"

response = requests.post(url)
print_highlight(response.text)

[2025-06-28 00:36:19] Cache flushed successfully!
[2025-06-28 00:36:19] INFO:     127.0.0.1:47120 - "POST /flush_cache HTTP/1.1" 200 OK


## Update Weights From Disk

Update model weights from disk without restarting the server. Only applicable for models with the same architecture and parameter size.

SGLang support `update_weights_from_disk` API for continuous evaluation during training (save checkpoint to disk and update weights from disk).


In [8]:
# successful update with same architecture and size

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct"}

response = requests.post(url, json=data)
print_highlight(response.text)
assert response.json()["success"] is True
assert response.json()["message"] == "Succeeded to update model weights."

[2025-06-28 00:36:19] Start update_weights. Load format=auto
[2025-06-28 00:36:19] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-06-28 00:36:19] Using model weights format ['*.safetensors']


[2025-06-28 00:36:20] No model.safetensors.index.json found in remote.
Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.80it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  4.79it/s]

[2025-06-28 00:36:20] Update weights end.
[2025-06-28 00:36:20] Cache flushed successfully!
[2025-06-28 00:36:20] INFO:     127.0.0.1:47134 - "POST /update_weights_from_disk HTTP/1.1" 200 OK


In [9]:
# failed update with different parameter size or wrong name

url = f"http://localhost:{port}/update_weights_from_disk"
data = {"model_path": "qwen/qwen2.5-0.5b-instruct-wrong"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(response_json)
assert response_json["success"] is False
assert response_json["message"] == (
    "Failed to get weights iterator: "
    "qwen/qwen2.5-0.5b-instruct-wrong"
    " (repository not found)."
)

[2025-06-28 00:36:20] Start update_weights. Load format=auto
[2025-06-28 00:36:20] Update engine weights online from disk begin. avail mem=41.50 GB
[2025-06-28 00:36:20] Failed to get weights iterator: qwen/qwen2.5-0.5b-instruct-wrong (repository not found).
[2025-06-28 00:36:20] INFO:     127.0.0.1:47150 - "POST /update_weights_from_disk HTTP/1.1" 400 Bad Request


In [10]:
terminate_process(server_process)

[2025-06-28 00:36:20] Child process unexpectedly failed with exitcode=9. pid=522845
[2025-06-28 00:36:20] Child process unexpectedly failed with exitcode=9. pid=522651


## Encode (embedding model)

Encode text into embeddings. Note that this API is only available for [embedding models](openai_api_embeddings.html#openai-apis-embedding) and will raise an error for generation models.
Therefore, we launch a new server to server an embedding model.

In [11]:
embedding_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-1.5B-instruct \
    --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 00:36:29] server_args=ServerArgs(model_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_path='Alibaba-NLP/gte-Qwen2-1.5B-instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Alibaba-NLP/gte-Qwen2-1.5B-instruct', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=37680, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=773707136, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_tim

[2025-06-28 00:36:33] Downcasting torch.float32 to torch.float16.


[2025-06-28 00:36:40] Downcasting torch.float32 to torch.float16.


[2025-06-28 00:36:41] Overlap scheduler is disabled for embedding models.
[2025-06-28 00:36:41] Downcasting torch.float32 to torch.float16.
[2025-06-28 00:36:41] Attention backend not set. Use fa3 backend by default.
[2025-06-28 00:36:41] Init torch distributed begin.


[2025-06-28 00:36:41] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-28 00:36:46] KV Cache is allocated. #tokens: 20480, K size: 0.27 GB, V size: 0.27 GB
[2025-06-28 00:36:46] Memory pool end. avail mem=57.72 GB


[2025-06-28 00:36:47] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.11 GB


[2025-06-28 00:36:47] INFO:     Started server process [525252]
[2025-06-28 00:36:47] INFO:     Waiting for application startup.
[2025-06-28 00:36:47] INFO:     Application startup complete.
[2025-06-28 00:36:47] INFO:     Uvicorn running on http://0.0.0.0:37680 (Press CTRL+C to quit)
[2025-06-28 00:36:47] INFO:     127.0.0.1:47482 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-28 00:36:48] INFO:     127.0.0.1:47484 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 00:36:48] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 00:36:49] INFO:     127.0.0.1:47498 - "POST /encode HTTP/1.1" 200 OK
[2025-06-28 00:36:49] The server is fired up and ready to roll!


In [12]:
# successful encode for embedding model

url = f"http://localhost:{port}/encode"
data = {"model": "Alibaba-NLP/gte-Qwen2-1.5B-instruct", "text": "Once upon a time"}

response = requests.post(url, json=data)
response_json = response.json()
print_highlight(f"Text embedding (first 10): {response_json['embedding'][:10]}")

[2025-06-28 00:36:52] Prefill batch. #new-seq: 1, #new-token: 4, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-28 00:36:52] INFO:     127.0.0.1:47508 - "POST /encode HTTP/1.1" 200 OK


In [13]:
terminate_process(embedding_process)

[2025-06-28 00:36:52] Child process unexpectedly failed with exitcode=9. pid=525598
[2025-06-28 00:36:52] Child process unexpectedly failed with exitcode=9. pid=525530


## v1/rerank (cross encoder rerank model)
Rerank a list of documents given a query using a cross-encoder model. Note that this API is only available for cross encoder model like [BAAI/bge-reranker-v2-m3](https://huggingface.co/BAAI/bge-reranker-v2-m3) with `attention-backend` `triton` and `torch_native`.


In [14]:
reranker_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path BAAI/bge-reranker-v2-m3 \
    --host 0.0.0.0 --disable-radix-cache --chunked-prefill-size -1 --attention-backend triton --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 00:37:02] server_args=ServerArgs(model_path='BAAI/bge-reranker-v2-m3', tokenizer_path='BAAI/bge-reranker-v2-m3', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='BAAI/bge-reranker-v2-m3', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=33251, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=352442311, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu

[2025-06-28 00:37:06] Downcasting torch.float32 to torch.float16.


[2025-06-28 00:37:15] Downcasting torch.float32 to torch.float16.


[2025-06-28 00:37:16] Overlap scheduler is disabled for embedding models.
[2025-06-28 00:37:16] Downcasting torch.float32 to torch.float16.
[2025-06-28 00:37:16] Init torch distributed begin.


[2025-06-28 00:37:17] Init torch distributed ends. mem usage=0.00 GB


...

[2025-06-28 00:37:22] KV Cache is allocated. #tokens: 20480, K size: 0.94 GB, V size: 0.94 GB
[2025-06-28 00:37:22] Memory pool end. avail mem=58.31 GB


[2025-06-28 00:37:23] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=8194, available_gpu_mem=57.73 GB


[2025-06-28 00:37:23] INFO:     Started server process [528033]
[2025-06-28 00:37:23] INFO:     Waiting for application startup.
[2025-06-28 00:37:23] INFO:     Application startup complete.
[2025-06-28 00:37:23] INFO:     Uvicorn running on http://0.0.0.0:33251 (Press CTRL+C to quit)


[2025-06-28 00:37:24] INFO:     127.0.0.1:38640 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-28 00:37:24] INFO:     127.0.0.1:38648 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 00:37:24] Prefill batch. #new-seq: 1, #new-token: 8, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 00:37:25] INFO:     127.0.0.1:38656 - "POST /encode HTTP/1.1" 200 OK
[2025-06-28 00:37:25] The server is fired up and ready to roll!


In [15]:
# compute rerank scores for query and documents

url = f"http://localhost:{port}/v1/rerank"
data = {
    "model": "BAAI/bge-reranker-v2-m3",
    "query": "what is panda?",
    "documents": [
        "hi",
        "The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.",
    ],
}

response = requests.post(url, json=data)
response_json = response.json()
for item in response_json:
    print_highlight(f"Score: {item['score']:.2f} - Document: '{item['document']}'")

[2025-06-28 00:37:29] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-28 00:37:29] Prefill batch. #new-seq: 1, #new-token: 43, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-28 00:37:29] INFO:     127.0.0.1:38670 - "POST /v1/rerank HTTP/1.1" 200 OK


In [16]:
terminate_process(reranker_process)

[2025-06-28 00:37:29] Child process unexpectedly failed with exitcode=9. pid=528535
[2025-06-28 00:37:29] Child process unexpectedly failed with exitcode=9. pid=528460


## Classify (reward model)

SGLang Runtime also supports reward models. Here we use a reward model to classify the quality of pairwise generations.

In [17]:
# Note that SGLang now treats embedding models and reward models as the same type of models.
# This will be updated in the future.

reward_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path Skywork/Skywork-Reward-Llama-3.1-8B-v0.2 --host 0.0.0.0 --is-embedding
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 00:37:37] server_args=ServerArgs(model_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_path='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Skywork/Skywork-Reward-Llama-3.1-8B-v0.2', chat_template=None, completion_template=None, is_embedding=True, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=37278, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=613191014, constrained_json_whitespace_pattern=None, watchdog_timeou

[2025-06-28 00:37:49] Overlap scheduler is disabled for embedding models.
[2025-06-28 00:37:49] Attention backend not set. Use flashinfer backend by default.
[2025-06-28 00:37:49] Init torch distributed begin.


[2025-06-28 00:37:50] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 00:37:51] Load weight begin. avail mem=77.99 GB


[2025-06-28 00:37:51] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


...

Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.21it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.56it/s]

[2025-06-28 00:37:54] Load weight end. type=LlamaForSequenceClassification, dtype=torch.bfloat16, avail mem=60.55 GB, mem usage=17.43 GB.


[2025-06-28 00:37:54] KV Cache is allocated. #tokens: 20480, K size: 1.25 GB, V size: 1.25 GB
[2025-06-28 00:37:54] Memory pool end. avail mem=57.76 GB


[2025-06-28 00:37:55] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=57.19 GB


[2025-06-28 00:37:55] INFO:     Started server process [531068]
[2025-06-28 00:37:55] INFO:     Waiting for application startup.
[2025-06-28 00:37:55] INFO:     Application startup complete.
[2025-06-28 00:37:55] INFO:     Uvicorn running on http://0.0.0.0:37278 (Press CTRL+C to quit)


[2025-06-28 00:37:56] INFO:     127.0.0.1:40186 - "GET /v1/models HTTP/1.1" 200 OK
[2025-06-28 00:37:56] INFO:     127.0.0.1:40196 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 00:37:56] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


In [18]:
from transformers import AutoTokenizer

PROMPT = (
    "What is the range of the numeric output of a sigmoid node in a neural network?"
)

RESPONSE1 = "The output of a sigmoid node is bounded between -1 and 1."
RESPONSE2 = "The output of a sigmoid node is bounded between 0 and 1."

CONVS = [
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE1}],
    [{"role": "user", "content": PROMPT}, {"role": "assistant", "content": RESPONSE2}],
]

tokenizer = AutoTokenizer.from_pretrained("Skywork/Skywork-Reward-Llama-3.1-8B-v0.2")
prompts = tokenizer.apply_chat_template(CONVS, tokenize=False)

url = f"http://localhost:{port}/classify"
data = {"model": "Skywork/Skywork-Reward-Llama-3.1-8B-v0.2", "text": prompts}

responses = requests.post(url, json=data).json()
for response in responses:
    print_highlight(f"reward: {response['embedding'][0]}")

[2025-06-28 00:39:00] Prefill batch. #new-seq: 2, #new-token: 136, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0
[2025-06-28 00:39:00] INFO:     127.0.0.1:40204 - "POST /encode HTTP/1.1" 200 OK
[2025-06-28 00:39:00] The server is fired up and ready to roll!


[2025-06-28 00:39:14] INFO:     127.0.0.1:55008 - "POST /classify HTTP/1.1" 200 OK


In [19]:
terminate_process(reward_process)

[2025-06-28 00:39:14] Child process unexpectedly failed with exitcode=9. pid=531662
[2025-06-28 00:39:14] Child process unexpectedly failed with exitcode=9. pid=531524


## Capture expert selection distribution in MoE models

SGLang Runtime supports recording the number of times an expert is selected in a MoE model run for each expert in the model. This is useful when analyzing the throughput of the model and plan for optimization.

*Note: We only print out the first 10 lines of the csv below for better readability. Please adjust accordingly if you want to analyze the results more deeply.*

In [20]:
expert_record_server_process, port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen1.5-MoE-A2.7B --host 0.0.0.0 --expert-distribution-recorder-mode stat"
)

wait_for_server(f"http://localhost:{port}")

[2025-06-28 00:39:21] server_args=ServerArgs(model_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_path='Qwen/Qwen1.5-MoE-A2.7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='Qwen/Qwen1.5-MoE-A2.7B', chat_template=None, completion_template=None, is_embedding=False, enable_multimodal=None, revision=None, impl='auto', host='0.0.0.0', port=39340, mem_fraction_static=0.874, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=733902885, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu

[2025-06-28 00:39:33] Attention backend not set. Use flashinfer backend by default.
[2025-06-28 00:39:33] Init torch distributed begin.


[2025-06-28 00:39:35] Init torch distributed ends. mem usage=0.00 GB


[2025-06-28 00:39:36] Load weight begin. avail mem=78.48 GB


[2025-06-28 00:39:36] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/8 [00:00<?, ?it/s]


...

[2025-06-28 00:39:43] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=47.28 GB


[2025-06-28 00:39:44] INFO:     Started server process [538146]
[2025-06-28 00:39:44] INFO:     Waiting for application startup.
[2025-06-28 00:39:44] INFO:     Application startup complete.
[2025-06-28 00:39:44] INFO:     Uvicorn running on http://0.0.0.0:39340 (Press CTRL+C to quit)


[2025-06-28 00:39:44] INFO:     127.0.0.1:54982 - "GET /v1/models HTTP/1.1" 200 OK


[2025-06-28 00:39:45] INFO:     127.0.0.1:54990 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-06-28 00:39:45] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0


[2025-06-28 00:39:46] Using default MoE kernel config. Performance might be sub-optimal! Config file not found at /public_sglang_ci/runner-l1a-gpu-1/_work/sglang/sglang/python/sglang/srt/layers/moe/fused_moe_triton/configs/triton_3_3_1/E=60,N=1408,device_name=NVIDIA_H100_80GB_HBM3.json, you can create them with https://github.com/sgl-project/sglang/tree/main/benchmark/kernels/fused_moe_triton


In [21]:
response = requests.post(f"http://localhost:{port}/start_expert_distribution_record")
print_highlight(response)

url = f"http://localhost:{port}/generate"
data = {"text": "What is the capital of France?"}

response = requests.post(url, json=data)
print_highlight(response.json())

response = requests.post(f"http://localhost:{port}/stop_expert_distribution_record")
print_highlight(response)

response = requests.post(f"http://localhost:{port}/dump_expert_distribution_record")
print_highlight(response)

[2025-06-28 00:40:01] Resetting ExpertDistributionRecorder...
[2025-06-28 00:40:01] INFO:     127.0.0.1:55006 - "POST /start_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-28 00:40:01] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 1, #queue-req: 0


[2025-06-28 00:40:02] INFO:     127.0.0.1:54992 - "POST /generate HTTP/1.1" 200 OK
[2025-06-28 00:40:02] The server is fired up and ready to roll!


[2025-06-28 00:40:03] Decode batch. #running-req: 1, #token: 46, token usage: 0.00, cuda graph: False, gen throughput (token/s): 2.28, #queue-req: 0


[2025-06-28 00:40:04] Decode batch. #running-req: 1, #token: 86, token usage: 0.00, cuda graph: False, gen throughput (token/s): 36.30, #queue-req: 0


[2025-06-28 00:40:05] Decode batch. #running-req: 1, #token: 126, token usage: 0.01, cuda graph: False, gen throughput (token/s): 35.89, #queue-req: 0


[2025-06-28 00:40:06] INFO:     127.0.0.1:53708 - "POST /generate HTTP/1.1" 200 OK


[2025-06-28 00:40:06] INFO:     127.0.0.1:34186 - "POST /stop_expert_distribution_record HTTP/1.1" 200 OK


[2025-06-28 00:40:07] Write expert distribution to /tmp/expert_distribution_recorder_1751071207.7641912.pt
[2025-06-28 00:40:07] Resetting ExpertDistributionRecorder...
[2025-06-28 00:40:07] INFO:     127.0.0.1:34194 - "POST /dump_expert_distribution_record HTTP/1.1" 200 OK


In [22]:
terminate_process(expert_record_server_process)

[2025-06-28 00:40:07] Child process unexpectedly failed with exitcode=9. pid=538597
[2025-06-28 00:40:07] Child process unexpectedly failed with exitcode=9. pid=538529
